In [ ]:
import warnings
warnings.filterwarnings("ignore")
from modules.functions import get_schedule,ids,create_home_and_away_simple_dataframe,BadStatusCodeError
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import pickle
ids = ids.loc[ids.link.notnull()].copy()
%cd Adjusting_Basketball_Stats

In [ ]:
master_df = pd.read_parquet('parquet_files/master_dataframe.gzip')
#already_scraped_ids = set(master_df.Game_ID.unique().tolist())
not_yet_played_ids = set()
with open('pickle_files/forfeitted_game_ids.pickle','rb') as f:
    forfeited_game_ids = pickle.load(f)
already_scraped_ids = set()

In [ ]:
counter = 0
bad_requests_counter = 0

for team in tqdm(ids.team.unique().tolist()):

    scrape_counter = 0
    time.sleep(np.random.randint(2,6))
    temp_master_dict = get_schedule(team).drop_duplicates(subset="GAME_ID").set_index("GAME_ID").to_dict(orient = 'index')
    quick_skip = True

    for game_id,sub_dict in temp_master_dict.items():

        game_id = int(game_id)

        if game_id in already_scraped_ids or game_id in not_yet_played_ids or game_id in forfeited_game_ids:
            continue
        
        # Attempt to get data, unless it hasn't been played - Log it as such, and move on
        try:
            home_df,away_df = create_home_and_away_simple_dataframe(game_id)
        except ValueError as e:
            not_yet_played_ids.add(game_id)
            continue
        except BadStatusCodeError as e:
            bad_requests_counter +=1
            if bad_requests_counter == 5:
                raise Exception("5 bad requests...")
            continue
        
        # Transform home and away dataframes
        home_df = home_df.reset_index().drop('level_1',axis = 1).rename(columns = {'level_0':"Team"})\
        .assign(Opponent = sub_dict['OPPONENT'].replace("@","").strip()).query("Player != 'Team'")\
        .assign(Game_ID = game_id).copy()

        away_df = away_df.reset_index().drop('level_1',axis = 1).rename(columns = {'level_0':'Team'.strip()})\
        .assign(Opponent = team).query("Player != 'Team'")\
        .assign(Game_ID = game_id).copy()

        final_df = pd.concat([home_df,away_df])
        final_df['Game_Date'] = sub_dict['DATE']

        master_df = pd.concat([master_df,final_df])
        already_scraped_ids.add(game_id)
        time_to_sleep = np.random.randint(7,14)

        counter +=1
        scrape_counter +=1
        if counter == 25:
            # Save every 25 iterations, just in case
            master_df.assign(Game_ID = master_df.Game_ID.astype(int)).to_parquet('parquet_files/master_dataframe.gzip',compression='gzip')
            print(master_df.Game_ID.nunique())
            counter = 0
        if scrape_counter >= 5:
            quick_skip = False
        
        if not quick_skip:
            # Sleep
            time.sleep(time_to_sleep)
        else:
            time.sleep(2)

    # Sleep
    if not quick_skip:
        time.sleep(np.random.randint(20,120))
    else:
        time.sleep(18)

master_df.reset_index(drop=True).to_parquet('parquet_files/master_dataframe.gzip',compression='gzip')

In [ ]:
master_df.to_parquet('parquet_files/master_dataframe.gzip',compression='gzip')
print(master_df.Game_ID.nunique())

with open('pickle_files/not_yet_played_games.pickle','wb') as f:
    pickle.dump(not_yet_played_ids,f)